In [123]:
from ast import literal_eval
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [105]:
# load dataset
credits = pd.read_csv('data/tmdb_credits.csv', header=0)
movies = pd.read_csv('data/tmdb_movies.csv', header=0)

In [161]:

indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
cred = credits.drop(columns=['title'])
# print(movies)
# take genres, spoken_languages, vectorize overview?, vectorize title, sort w popularity/rating,
# def weighted_rating(x, m=m, C=C):
#     v = x['vote_count']
#     R = x['vote_average']
#     # Calculation based on the IMDB formula
#     return (v/(v+m) * R) + (m/(m+v) * C)


# C = movies['vote_average'].mean()
# print(C)
# q_movies = movies.copy()
# q_movies.shape
def extract_words(row):
    features = ['keywords', 'genres', 'director']
    ret = ''
    for f in features:
        ret+=' '.join(row[f])+ " "
    return ret

def process_str(item, key):
    return str.lower(item[key].replace(" ", "")) 

def process_col(col, key):
    return data[col].fillna('[]').apply(literal_eval).apply(lambda x: [process_str(i, key) for i in x] if x else [])

data = movies.drop(columns = ['budget', 'homepage', 'original_title', 'production_companies', 'revenue', 'status', 'spoken_languages'])
data = data.set_index('id').join(cred.set_index('movie_id'))

data['genres'] = process_col('genres', 'name')
data['keywords'] = process_col('keywords', 'name')
data['director'] = process_col('crew', 'name')
data['cast'] = process_col('cast', 'name')
data['extract'] = data.apply(extract_words, axis = 1)


In [169]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit(data['extract'])
print(count_matrix)
# cos_sim = cosine_similarity(count_matrix, count_matrix)

CountVectorizer(stop_words='english')


In [165]:
def get_recommendations(title, cosine_sim=cos_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]

In [166]:
get_recommendations('Titanic')

id
36955                      True Lies
280       Terminator 2: Judgment Day
19995                         Avatar
679                           Aliens
36046         The Work and the Glory
14834              Not Easily Broken
361505         Me You and Five Bucks
77332                        Urbania
86549                         August
18701                    Being Julia
Name: title, dtype: object